# Ensemble backward stepwise and KNN regression

For the period 2011-2018, ensembles the predictions of backward stepwise (MultiLLR), KNN autoregression (AutoKNN), and/or debiased CFSv2 and saves the skills of both the individual methods and the ensembles.

In [ ]:
# Autoreload packages that are modified
%load_ext autoreload
%autoreload 2

# Load relevant packages
import numpy as np
import pandas as pd
from sklearn import *
import sys
import subprocess
from datetime import datetime, timedelta
import netCDF4
import time
import pickle
from functools import partial
import os

if os.path.basename(os.getcwd()) == "experiments":
    os.chdir(os.path.join("..",".."))

# Adds 'experiments' folder to path to load experiments_util
sys.path.insert(0, 'src/experiments')
# Load general utility functions
from experiments_util import *
# Load functionality for fitting and predicting
from fit_and_predict import *
# Load functionality for evaluation
from skill import *

In [ ]:
# User inputs
gt_id = "contest_tmp2m" # "contest_tmp2m" or "contest_precip"
target_horizon = "56w"  # "34w" or "56w"

In [ ]:
# Location of backward stepwise results
# temp 34
if gt_id == "contest_tmp2m" and target_horizon == "34w":
    results_dir="results/regression/2011-2018/contest_tmp2m_34w/backward_stepwise/margin56-mean-4762873656607921567/"
# temp 56
if gt_id == "contest_tmp2m" and target_horizon == "56w":
    results_dir="results/regression/2011-2018/contest_tmp2m_56w/backward_stepwise/margin56-mean-6754054503168589598/"
# precip 34
if gt_id == "contest_precip" and target_horizon == "34w":
    results_dir="results/regression/2011-2018/contest_precip_34w/backward_stepwise/margin56-mean-8442516410103942925/"
# precip 56
if gt_id == "contest_precip" and target_horizon == "56w":
    results_dir="results/regression/2011-2018/contest_precip_56w/backward_stepwise/margin56-mean-3061203412216199752/"

#---------------
# Load backward stepwise results
#---------------

# Choose which model to evaluate along the stepwise path:
# k=1 means the first model on the path, k=2 means the second model on the path,
# k=-1 means the last model on the path
k=-1

all_files = [f for f in os.listdir(results_dir) if f.endswith('h5')]
preds = pd.DataFrame()
for f in all_files:
    d = pd.read_hdf(os.path.join(results_dir, f))
    if d.shape[0] > 0:
        ncol = d.shape[1]
        pred_col = -1 if k==-1 else min(4+k, ncol-1)
        d['pred'] = d.iloc[:,pred_col].tolist()
        d['pred'] = preprocessing.normalize(d['pred'].values.reshape(1,-1))[0,:]
        preds = pd.concat([preds, d[['lat','lon','start_date','truth','clim','pred']]])

In [ ]:
# Save the skills of backward stepwise alone
skills = get_col_skill(preds, 'pred', 'truth', time_average=False)
skills = pd.DataFrame(skills).reset_index()
skills.columns = ['start_date', 'skill']
backward_skill_file = os.path.join('results','skills',
                         'backward_stepwise',
                         'skill-{}-{}-margin56-mean.h5'.format(gt_id, target_horizon))
# Create results subdirectories if they don't exist
if not os.path.isdir(os.path.dirname(backward_skill_file)):
    os.makedirs(os.path.dirname(backward_skill_file))
skills.to_hdf(backward_skill_file, key="data", mode="w")

In [ ]:
# Location of KNN regression results
knn_filename_string='{}-{}-days60-early{}'.format(gt_id, target_horizon, 323 if target_horizon == '56w' else 337)
# temp 34
if gt_id == "contest_tmp2m" and target_horizon == "34w":
    knn_file='results/regression/2011-2018/contest_tmp2m_34w/3801171228559066888/preds-{}.h5'.format(knn_filename_string)
# temp 56
if gt_id == "contest_tmp2m" and target_horizon == "56w":
    knn_file='results/regression/2011-2018/contest_tmp2m_56w/292106474723564147/preds-{}.h5'.format(knn_filename_string)
# precip 34
if gt_id == "contest_precip" and target_horizon == "34w":
    knn_file='results/regression/2011-2018/contest_precip_34w/1154060661897479144/preds-{}.h5'.format(knn_filename_string)
# precip 56
if gt_id == "contest_precip" and target_horizon == "56w":
    knn_file='results/regression/2011-2018/contest_precip_56w/74648686601095817/preds-{}.h5'.format(knn_filename_string)

#---------------
# Load KNN regression results
#---------------
knn_preds = pd.read_hdf(knn_file)
knn_preds.reset_index(drop=True, inplace=True)

# Normalize forecast predictions
knn_preds['forecast'] = knn_preds.groupby(knn_preds.start_date)['forecast'].transform(lambda x: x/np.sqrt(sum(x**2)))

In [ ]:
# Save the skills of the KNN regression alone
skills = get_col_skill(knn_preds, 'forecast', 'truth', time_average=False)
skills = pd.DataFrame(skills).reset_index()
skills.columns = ['start_date', 'skill']
knn_skill_file = os.path.join('results/skills/',
                         'knn_target_regression',
                         'skill-{}.h5'.format(knn_filename_string))
# Create results subdirectories if they don't exist
if not os.path.isdir(os.path.dirname(knn_skill_file)):
    os.makedirs(os.path.dirname(knn_skill_file))
skills.to_hdf(knn_skill_file, key="data", mode="w")

In [ ]:
# Location of debiased CFSv2 regression results
# temp 34
if gt_id == "contest_tmp2m" and target_horizon == "34w":
    cfs_file='results/skills/cfsv2/debiased_cfsv2_tmp2m_34_forecast_2011-2018.h5'
    cfs_anom = 'debiased_cfsv2_tmp2m_anom'
# temp 56
if gt_id == "contest_tmp2m" and target_horizon == "56w":
    cfs_file='results/skills/cfsv2/debiased_cfsv2_tmp2m_56_forecast_2011-2018.h5'
    cfs_anom = 'debiased_cfsv2_tmp2m_anom'
# precip 34
if gt_id == "contest_precip" and target_horizon == "34w":
    cfs_file='results/skills/cfsv2/debiased_cfsv2_precip_34_forecast_2011-2018.h5'
    cfs_anom = 'debiased_cfsv2_precip_anom'
# precip 56
if gt_id == "contest_precip" and target_horizon == "56w":
    cfs_file='results/skills/cfsv2/debiased_cfsv2_precip_56_forecast_2011-2018.h5'
    cfs_anom = 'debiased_cfsv2_precip_anom'

#---------------
# Load debiased CFSv2 results
#---------------
cfs_preds = pd.read_hdf(cfs_file)
# Normalize forecast predictions
cfs_preds[cfs_anom] = cfs_preds.groupby(cfs_preds.start_date)[cfs_anom].transform(lambda x: x/np.sqrt(sum(x**2)))

In [ ]:
# Save the skills of debiased CFSv2 regression alone
skills = get_col_skill(pd.merge(cfs_preds, preds[['lat','lon','start_date','truth']],
                                on=['lat','lon','start_date'], how='left'), 
                       cfs_anom, 'truth', time_average=False)
skills = pd.DataFrame(skills).reset_index()
skills.columns = ['start_date', 'skill']
cfs_skill_file = os.path.join('results/skills/',
                             'debiased_cfsv2',
                             'skill-{}-{}.h5'.format(gt_id, target_horizon))
# Create results subdirectories if they don't exist
if not os.path.isdir(os.path.dirname(cfs_skill_file)):
    os.makedirs(os.path.dirname(cfs_skill_file))
skills.to_hdf(cfs_skill_file, key="data", mode="w")

In [ ]:
# Compute predictions for each of the three following ensembles and save skills to file
# 'backward_stepwise_and_knn': equal-weighted ensemble of backward stepwise and KNN autoregression
# 'backward_stepwise_knn_debiased_cfsv2': equal-weighted ensemble of backward stepwise, KNN autoregression,
#    and reconstructed debiased CFSv2
# 'knn_debiased_cfsv2': equal-weighted ensemble of KNN autoregression and reconstructed debiased CFSv2

ensemble_preds = pd.merge(preds, knn_preds[['lat','lon','start_date','forecast']], on=['lat','lon','start_date'], how='left')
ensemble_preds = pd.merge(ensemble_preds, cfs_preds[['lat','lon','start_date',cfs_anom]], on=['lat','lon','start_date'], how='left')

for ensemble in ['backward_stepwise_knn_debiased_cfsv2', 'knn_debiased_cfsv2', 'backward_stepwise_and_knn']:
    if ensemble == 'backward_stepwise_knn_debiased_cfsv2':
        file_name = os.path.join('results/skills/',
                                 'ensemble_backward_stepwise_knn_debiased_cfsv2',
                                 'skill-{}-{}-margin56-mean.h5'.format(gt_id, target_horizon))
    if ensemble == 'knn_debiased_cfsv2':
        file_name = os.path.join('results/skills/',
                                 'ensemble_knn_debiased_cfsv2',
                                 'skill-{}-{}.h5'.format(gt_id, target_horizon))
    if ensemble == 'backward_stepwise_and_knn':
        file_name = os.path.join('results/skills/',
                                 'ensemble_backward_stepwise_and_knn',
                                 'skill-{}-{}-margin56-mean.h5'.format(gt_id, target_horizon))

    # Create results subdirectories if they don't exist
    if not os.path.isdir(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))

    # Compute the ensemble predictions
    if ensemble == 'backward_stepwise_knn_debiased_cfsv2':
        ensemble_preds['ensemble_pred'] = ensemble_preds['pred']/3 + ensemble_preds['forecast']/3 + ensemble_preds[cfs_anom]/3
    if ensemble == 'knn_debiased_cfsv2':
        ensemble_preds['ensemble_pred'] = ensemble_preds['forecast']/2 + ensemble_preds[cfs_anom]/2
    if ensemble == 'backward_stepwise_and_knn':
        ensemble_preds['ensemble_pred'] = 0.5*ensemble_preds['pred'] + 0.5*ensemble_preds['forecast']
        
    # Compute skills and save to file
    skills = get_col_skill(ensemble_preds, 'ensemble_pred', 'truth', time_average=False)
    skills = pd.DataFrame(skills).reset_index()
    skills.columns = ['start_date', 'skill']
    skills.to_hdf(file_name, key="data", mode="w")
    
    # Summary of average yearly skill achieved by ensemble
    print ensemble
    for year in range(2011, 2018):
        skills_year = skills[(skills.start_date >= get_target_date(str(year)+"0418", target_horizon)) & 
                             (skills.start_date <= get_target_date(str(year+1)+"0417", target_horizon))]
        print str(year) + ": " + str(skills_year.mean().values[0])
    
    print "Overall: " + str(skills.mean().values[0])